<a href="https://colab.research.google.com/github/joestanis/USGS-SAC-25-12553801-DE-RM/blob/main/JoeStanis_USGS_SeismicData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervisory Computer Scientist, Vacancy Announcement USGS-SAC-25-12553801-DE-RM

## Purpose

A project designed and developed using specific objectives, emphasizing code efficiency, readability, and adherence to best practices.

## Import the mseed files.

In [ ]:
print('TODO')

## Create a database with proper normal form and constraints.

In [ ]:
print('TODO')

## Import the data into the newly created database.

In [ ]:
print('TODO')

## Create a visualization for the data.

In [ ]:
print('TODO')